In [1]:
import pandas as pd

In [2]:
DATA_DIR = './data/'

In [3]:
df = pd.read_csv('{}Train_Data.csv'.format(DATA_DIR))
df.head()

,date,number of rooms,security level of the community,residence space,building space,noise level,waterfront,view,air quality level,aboveground space,...,building year,decoration year,district,city,zip code,region,exchange rate,unit price of residence space,unit price of building space,total cost
0,2014/5/2 0:00,5,2.50,2820,67518,2.0,0,0,3,2820,...,1979,2014,23525 SE 32nd Way,Issaquah,WA 98029,USA,6.784830,11.886409,0.977028,NaN
1,2014/5/2 0:00,2,1.00,1210,9400,1.0,0,0,2,1210,...,1949,0,7542 21st Ave SW,Seattle,WA 98106,USA,6.243129,25.681414,0.698603,NaN
2,2014/5/2 0:00,4,2.50,2200,9397,2.0,0,0,3,2200,...,1987,2000,5214 S 292nd St,Auburn,WA 98001,USA,6.010127,16.921174,0.238882,NaN
3,2014/5/2 0:00,3,1.50,1200,9720,1.0,0,0,4,1200,...,1965,0,14034 SE 201st St,Kent,WA 98042,USA,7.560375,19.632230,0.876178,NaN
4,2014/5/2 0:00,3,1.75,1370,5858,1.0,0,0,3,1370,...,1987,2000,1605 S 245th Pl,Des Moines,WA 98198,USA,6.543941,28.205930,0.132472,NaN


# split dataset

In [4]:
data = df.to_numpy().tolist()
print('{}, {}'.format(len(data), len(data[0])))

4000, 21


In [5]:
size = len(data)

In [6]:
NUM_MAPPER = 5
NUM_REDUCER = 2
CHUNK_SIZE = size // NUM_MAPPER

In [7]:
chunks = [data[i: i+CHUNK_SIZE] for i in range(0, size, CHUNK_SIZE)]
print('{}, {}'.format(len(chunks), len(chunks[0])))

5, 800


# define map & reduce

In [8]:
def mapper(k, v):
    ret = list()
    for i,r in enumerate(v):
        key = k*CHUNK_SIZE+i
        exchange_rate = r[17]
        ret.append((key, exchange_rate * (r[3]*r[18])))    # total cost in residence
        ret.append((key, exchange_rate * (r[4]*r[19])))    # total cost in building
    return ret


def reducer(k, v):
    return k, sum(v)    # total cost of residence and building



In [9]:
middle_result = [mapper(k,v) for k,v in enumerate(chunks)]
middle_result

[[(0, 227425.27644498547),
  (0, 447574.72365968407),
  (1, 194002.17910038662),
  (1, 40997.82088796767),
  (2, 223736.50149504634),
  (2, 13491.355629791296),
  (3, 178112.4276469212),
  (3, 64387.57228799974),
  (4, 252871.76906822),
  (4, 5078.230893679583),
  (5, 256904.52711611163),
  (5, 3095.472895693038),
  (6, 200001.75835500288),
  (6, 74998.2416630014),
  (7, 269037.6298750002),
  (7, 15962.370144068793),
  (8, 272738.98827403807),
  (8, 14461.011733419578),
  (9, 294286.1419050498),
  (9, 713.858132864814),
  (10, 275658.8737217654),
  (10, 24341.12629240134),
  (11, 307279.13794740377),
  (11, 1220.8620511804834),
  (12, 283165.22323518683),
  (12, 31834.776716038206),
  (13, 325313.5544717195),
  (13, 9686.445593395298),
  (14, 322876.84645931417),
  (14, 12123.153516977423),
  (15, 297912.0630595378),
  (15, 44087.93704505871),
  (16, 340677.4007884351),
  (16, 24322.599177082146),
  (17, 345433.99602891994),
  (17, 22066.003940534156),
  (18, 361861.6843011309),
  (18,

## merge identical keys

In [10]:
from itertools import chain
total_middle_result = list(chain(*middle_result))
kv_pairs = [(total_middle_result[i][0], [total_middle_result[i][1], total_middle_result[i+1][1]]) for i in range(0, len(total_middle_result), 2)]
kv_pairs

[(0, [227425.27644498547, 447574.72365968407]),
 (1, [194002.17910038662, 40997.82088796767]),
 (2, [223736.50149504634, 13491.355629791296]),
 (3, [178112.4276469212, 64387.57228799974]),
 (4, [252871.76906822, 5078.230893679583]),
 (5, [256904.52711611163, 3095.472895693038]),
 (6, [200001.75835500288, 74998.2416630014]),
 (7, [269037.6298750002, 15962.370144068793]),
 (8, [272738.98827403807, 14461.011733419578]),
 (9, [294286.1419050498, 713.858132864814]),
 (10, [275658.8737217654, 24341.12629240134]),
 (11, [307279.13794740377, 1220.8620511804834]),
 (12, [283165.22323518683, 31834.776716038206]),
 (13, [325313.5544717195, 9686.445593395298]),
 (14, [322876.84645931417, 12123.153516977423]),
 (15, [297912.0630595378, 44087.93704505871]),
 (16, [340677.4007884351, 24322.599177082146]),
 (17, [345433.99602891994, 22066.003940534156]),
 (18, [361861.6843011309, 20638.3157918143]),
 (19, [384647.0649096422, 352.93504559405915]),
 (20, [358200.5066774682, 44799.4933201515]),
 (21, [35

## shuffle middle results

In [11]:
import random

In [12]:
random.seed(42)
mixed_result = kv_pairs
random.shuffle(mixed_result)
mixed_result

[(1823, [532299.8435007674, 2700.156517908027]),
 (228, [210479.97703142223, 104470.02290638458]),
 (3220, [352012.9195164896, 107987.08053142963]),
 (3202, [277885.4429397947, 87114.55708578079]),
 (2729, [423237.9954723851, 13262.004453850775]),
 (2784, [1187822.3841285682, 52177.6160213699]),
 (2938, [225332.10851401062, 26667.891507591827]),
 (3103, [471256.7163667909, 10743.283562803328]),
 (3693, [338106.58395359543, 13892.416053122271]),
 (2798, [464330.3446194916, 35669.65538007028]),
 (2588, [579710.3900541423, 70289.6099918738]),
 (2476, [434644.4323552253, 31305.56771331866]),
 (3742, [632048.1947261184, 256501.80531062288]),
 (2951, [282917.2693440823, 42032.730642476876]),
 (1198, [146385.12688535132, 48614.873148443075]),
 (1136, [312338.53993845737, 22661.460103196496]),
 (1596, [790896.7137215515, 29103.28625871546]),
 (3854, [1077089.996269124, 37910.00393568087]),
 (2812, [220183.3687075556, 13316.6313095795]),
 (2142, [524725.8621840762, 5274.137827439917]),
 (668, [

In [13]:
MIDDLE_CHUNK_SIZE = len(mixed_result) // NUM_REDUCER
partial_middle_result = [mixed_result[i: i + MIDDLE_CHUNK_SIZE] for i in range(0, len(mixed_result), MIDDLE_CHUNK_SIZE)]
print('{}, {}'.format(len(partial_middle_result), len(partial_middle_result[0])))


2, 2000


In [14]:
result = [reducer(*kv) for p in partial_middle_result for kv in p]
len(result)

4000

In [15]:
r = [e[1] for e in sorted(result, key=lambda x: x[0])]
r

[675000.0001046695,
 234999.99998835428,
 237227.85712483764,
 242499.99993492092,
 257949.99996189956,
 260000.00001180466,
 275000.00001800427,
 285000.000019069,
 287200.0000074576,
 295000.0000379146,
 300000.00001416676,
 308499.9999985843,
 314999.999951225,
 335000.00006511476,
 334999.9999762916,
 342000.0001045965,
 364999.9999655172,
 367499.9999694541,
 382500.0000929452,
 384999.9999552363,
 402999.9999976197,
 407500.00010218547,
 419000.00003359845,
 419999.9999640184,
 430000.0000039196,
 435000.0000309359,
 436110.0000488888,
 437500.00013730675,
 439950.00002122903,
 439999.99993771315,
 445700.0000409877,
 459990.0000360225,
 463000.00000052707,
 470000.00000713434,
 482000.00010247354,
 491499.9999503717,
 494999.9999533413,
 499949.9999487451,
 550000.0000965799,
 550000.0000413094,
 554999.9998967744,
 588500.0000169227,
 603999.9998611084,
 612499.9998769831,
 614999.9999597889,
 624999.9997949207,
 625000.000010636,
 626000.000066018,
 630000.0001182908,
 640000.

# save result

In [16]:
s = pd.Series(r, dtype='float')

In [17]:
df['total cost'] = s

In [18]:
df.head()

,date,number of rooms,security level of the community,residence space,building space,noise level,waterfront,view,air quality level,aboveground space,...,building year,decoration year,district,city,zip code,region,exchange rate,unit price of residence space,unit price of building space,total cost
0,2014/5/2 0:00,5,2.50,2820,67518,2.0,0,0,3,2820,...,1979,2014,23525 SE 32nd Way,Issaquah,WA 98029,USA,6.784830,11.886409,0.977028,675000.000105
1,2014/5/2 0:00,2,1.00,1210,9400,1.0,0,0,2,1210,...,1949,0,7542 21st Ave SW,Seattle,WA 98106,USA,6.243129,25.681414,0.698603,234999.999988
2,2014/5/2 0:00,4,2.50,2200,9397,2.0,0,0,3,2200,...,1987,2000,5214 S 292nd St,Auburn,WA 98001,USA,6.010127,16.921174,0.238882,237227.857125
3,2014/5/2 0:00,3,1.50,1200,9720,1.0,0,0,4,1200,...,1965,0,14034 SE 201st St,Kent,WA 98042,USA,7.560375,19.632230,0.876178,242499.999935
4,2014/5/2 0:00,3,1.75,1370,5858,1.0,0,0,3,1370,...,1987,2000,1605 S 245th Pl,Des Moines,WA 98198,USA,6.543941,28.205930,0.132472,257949.999962


In [20]:
# df.to_csv('{}Train_data_completed.csv'.format(DATA_DIR), index=False)